In [1]:
import numpy as np
import pandas as pd
import pickle
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

## Build Model

In [2]:
df_train = pd.read_csv("data/train_clean.csv", header=0, index_col=0)
X = df_train.drop(labels=["SalePrice"], axis=1)
y = df_train["SalePrice"]

In [3]:
# sm model
X_sm = sm.add_constant(X)
model_sm = sm.OLS(y, X_sm).fit()
print(model_sm.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     64.87
Date:                Thu, 15 Nov 2018   Prob (F-statistic):               0.00
Time:                        21:12:32   Log-Likelihood:                -16567.
No. Observations:                1460   AIC:                         3.365e+04
Df Residuals:                    1200   BIC:                         3.503e+04
Df Model:                         259                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -2.782e+

# Feature Selection

In [4]:
estimator = linear_model.LinearRegression()
rfe = RFE(estimator, n_features_to_select=110, step=1)
rfe.fit(X.values, y.values)
X = df_train[X.columns[rfe.support_]]

/Users/adamMac/miniconda3/envs/ml_blog/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor  
variables = list(X.columns)
vif = {variable:variance_inflation_factor(exog=X.values, exog_idx=ix) for ix,variable in enumerate(list(X.columns))}
vif

/Users/adamMac/miniconda3/envs/ml_blog/lib/python3.6/site-packages/statsmodels/stats/outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


{'OverallQual': 3.721538290940388,
 'BsmtFullBath': 1.7001141833797584,
 'FullBath': 2.591271575663198,
 'HalfBath': 2.4100513745096834,
 'KitchenAbvGr': 2.2033792345640437,
 'TotRmsAbvGrd': 3.0137181066844283,
 'Fireplaces': 1.5880190022942413,
 'GarageCars': 2.0351033189225003,
 'MSSubClass_20': 4.624124078572873,
 'MSSubClass_30': 2.289636511776471,
 'MSSubClass_40': 1.5039476594264676,
 'MSSubClass_50': 2.998150278720479,
 'MSSubClass_60': 5.228143935222247,
 'MSSubClass_70': 2.324589208909088,
 'MSSubClass_75': 1.589455104801475,
 'MSSubClass_120': 2.2246503184777002,
 'MSSubClass_160': 2.174522228679098,
 'MSSubClass_180': 1.2709356869588,
 'MSZoning_C (all)': 1.2136614128689156,
 'MSZoning_FV': 1.416003492232818,
 'Street_Pave': 1.208961090599696,
 'LotShape_IR1': 7.960597335632764,
 'LotShape_Reg': 8.681774036145285,
 'Utilities_NoSeWa': 1.0316939468458106,
 'LotConfig_CulDSac': 1.3060255175670832,
 'LotConfig_FR2': 1.0876103661679986,
 'LotConfig_FR3': 1.1431200269750208,
 'Ne

In [6]:
high_vif_cols = []
for i in vif:
    vif_threshold = 5
    if vif[i] > vif_threshold:
        high_vif_cols.append(i)
X = X.drop(labels=high_vif_cols, axis=1)

# K-Fold Cross Validation

In [7]:
X_cv = X.values
y_cv = y.values

kf = KFold(n_splits=10)
kf.get_n_splits(X_cv)
cv_results = pd.DataFrame(columns=[
    "test_idx", "R2 train", "RMSE train", "MAE test", 
    "R2 test", "RMSE test", "MAE test"])
for train_index, test_index in kf.split(X_cv):
    X_train, X_test = X_cv[train_index], X_cv[test_index]
    y_train, y_test = y_cv[train_index], y_cv[test_index]
    model = linear_model.LinearRegression()
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    # remove <0 prices
    npvec_zero = np.vectorize(lambda x: max(min(y_train_pred), x))
    y_train_pred = npvec_zero(y_train_pred)
    y_test_pred = npvec_zero(y_test_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mse = mean_squared_error(y_train, y_train_pred)
    rmse_train = mse ** (0.5)
    r2_train = r2_score(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    mse = mean_squared_error(y_test, y_test_pred)
    rmse_test = mse ** (0.5)
    r2_test = r2_score(y_test, y_test_pred)
    cv_results.loc[len(cv_results)] = ["{} - {}".format(min(test_index), max(test_index)), 
                  r2_train, rmse_train, mae_train,
                  r2_test, rmse_test, mae_test]
cv_results

,test_idx,R2 train,RMSE train,MAE test,R2 test,RMSE test,MAE test
0,0 - 145,0.884578,27363.330938,18585.333400,0.874723,24106.779528,18317.230093
1,146 - 291,0.885699,26828.439841,18188.420480,0.813826,34307.767393,20383.168097
2,292 - 437,0.883746,27082.825178,18433.653649,0.881563,27271.494068,19123.627356
3,438 - 583,0.893519,25786.248996,18057.974058,0.797711,37256.624004,22605.315837
4,584 - 729,0.883332,26404.007909,18045.117105,0.883220,32876.660895,22057.934859
5,730 - 875,0.883928,27153.239787,18501.684915,0.876816,26946.569112,17748.778658
6,876 - 1021,0.884724,27223.604138,18547.311914,0.873417,25515.046940,18467.208854
7,1022 - 1167,0.886837,27006.209632,18408.284330,0.838832,28354.546186,20227.092051
8,1168 - 1313,0.885217,26463.989134,18191.813947,0.700582,49203.259394,24084.375898
9,1314 - 1459,0.885090,27165.426329,18506.970845,0.863232,26766.944331,18842.992505


In [8]:
# save output
model = linear_model.LinearRegression()
model.fit(X, y)
pickle.dump(model, open("estimator_linreg.pkl", "wb"))
pickle.dump(X.columns, open("columns_model_only.pkl", "wb"))

In [9]:
# sm model - final summary
X_sm = sm.add_constant(X)
model_sm = sm.OLS(y, X_sm).fit()
print(model_sm.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                     133.7
Date:                Thu, 15 Nov 2018   Prob (F-statistic):               0.00
Time:                        21:12:48   Log-Likelihood:                -16969.
No. Observations:                1460   AIC:                         3.410e+04
Df Residuals:                    1380   BIC:                         3.452e+04
Df Model:                          79                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 4.758e+04 